### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school = "Resources/schools_complete.csv"
student = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school = pd.read_csv(school)
student = pd.read_csv(student)

# Combine the data into a single dataset.  
complete= pd.merge(school, student, how="left", on=["school_name", "school_name"])
#have a look of overview of the combined table
complete.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [10]:
#using unique for the getting the unique  school name , then use len to get the counts 
school_count= len(complete["school_name"].unique())
                  
#using student ID as key value to count the total students , as name might be duplicated 
total_students=len(complete["Student ID"].unique())


#sum up budget school table, instead of complete table, as complete table will adds up budget for the same school multiple times  
total_budget = school["budget"].sum()

#average math and reading score using mean function
average_math=complete["math_score"].mean()
average_reading=complete["reading_score"].mean()


#get total student counts who has score higher or equal to 70 , using boolean, when condition being meet, true=1 , sum up
# then divide by total students to get the pass rate 
math_pass_students=complete['math_score']>=70
math_pass_rate=(math_pass_students.sum()/total_students)*100
#print(math_pass_rate)

reading_pass_students=complete['reading_score']>=70
reading_pass_rate=(reading_pass_students.sum()/total_students)*100
#print(reading_pass_rate)

#meet both criterias, as two boolean condition, so count the results on new df, then divide by total students 
all_pass_students=complete[(complete['math_score']>=70) & (complete['reading_score']>=70)]
all_pass_rate=(all_pass_students["Student ID"].count()/total_students)*100
#print(all_pass_rate)

#summary 
District_Summary = pd.DataFrame({"Total Schools": [school_count],
                        "Total Students": [total_students],
                        "Total Budget": [total_budget],
                        "Average Math Score": [average_math],
                        "Average Reading Score": [average_reading],
                        "% Passing Math": [math_pass_rate],
                        "% Passing Reading": [reading_pass_rate],
                        "% Overall Passing": [all_pass_rate],
                            })

#format
District_Summary["Total Students"]=District_Summary["Total Students"].map("{:,}".format)
District_Summary["Total Budget"]=District_Summary["Total Budget"].map("${:,.2f}".format)
District_Summary["Average Math Score"]=round(District_Summary["Average Math Score"],2)
District_Summary["Average Reading Score"]=round(District_Summary["Average Reading Score"],2)
District_Summary["% Passing Math"]=District_Summary["% Passing Math"].map("{:.2f}%".format)
District_Summary["% Passing Reading"]=District_Summary["% Passing Reading"].map("{:.2f}%".format)
District_Summary["% Overall Passing"]=District_Summary["% Overall Passing"].map("{:.2f}%".format)
District_Summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary
* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [17]:
#adding budget per stu into school table 
school["Per Student Budget"]=school['budget']/school['size']

#statistic analysis on student csv
average_math_score=student.groupby('school_name')['math_score'].mean()
average_reading_score=student.groupby('school_name')['reading_score'].mean()

math_pass_rate=student[student['math_score'] >= 70].groupby('school_name')['Student ID'].count()/student.groupby('school_name')['Student ID'].count() * 100
reading_pass_rate=student[student['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/student.groupby('school_name')['Student ID'].count() * 100
total_pass_rate=student[(student['math_score']>=70) & (student['reading_score']>=70)].groupby('school_name')['Student ID'].count()/student.groupby('school_name')['Student ID'].count() * 100

#concating all mathmatical results
Student_Summary = pd.DataFrame({"Average Math Score": average_math_score,
                        "Average Reading Score": average_reading_score,
                        "Average Reading Score": average_reading_score,
                        "% Passing Math": math_pass_rate,
                        "% Passing Reading": reading_pass_rate,
                        "% Overall Passing": total_pass_rate,        
                            })
#merge school table with student summary
School_Summary = pd.merge(school, Student_Summary, on="school_name")
#rename column 
School_Summary=School_Summary.rename(columns={"type":"School Type", "size":"Total Students","budget":"Total School Budget"})
#set school name as index 
indext_school_summary=School_Summary.set_index('school_name')
#remove school ID column 
Final_school_summary=indext_school_summary.iloc[:,1:]

#format
Final_school_summary["Total Students"]=Final_school_summary["Total Students"].map("{:,}".format)
Final_school_summary["Per Student Budget"]=Final_school_summary["Per Student Budget"].map("${:,.2f}".format)
Final_school_summary["Total School Budget"]=Final_school_summary["Total School Budget"].map("${:,.2f}".format)
Final_school_summary["Average Math Score"]=round(Final_school_summary["Average Math Score"],2)
Final_school_summary["Average Reading Score"]=round(Final_school_summary["Average Reading Score"],2)
Final_school_summary["% Passing Math"]=Final_school_summary["% Passing Math"].map("{:.2f}%".format)
Final_school_summary["% Passing Reading"]=Final_school_summary["% Passing Reading"].map("{:.2f}%".format)
Final_school_summary["% Overall Passing"]=Final_school_summary["% Overall Passing"].map("{:.2f}%".format)

Final_school_summary



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.36,83.73,93.87%,95.85%,89.89%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [18]:
Final_school_summary.sort_values(by=['% Overall Passing'],ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [19]:
Final_school_summary.sort_values(by=['% Overall Passing'],ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [20]:
#using condition, getting average value on reading score  
ninth_grade = student[student['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_grade = student[student['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_grade = student[student['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_grade = student[student['grade'] == '12th'].groupby('school_name')["math_score"].mean()

Math_Score_by_Grade = pd.DataFrame({"9th": ninth_grade,
                                    "10th": tenth_grade,
                                    "11th": eleventh_grade,
                                    "12th": twelfth_grade,        
                            })
#format
Math_Score_by_Grade["9th"]=Math_Score_by_Grade["9th"].map("{:.2f}".format)
Math_Score_by_Grade["10th"]=Math_Score_by_Grade["10th"].map("{:.2f}".format)
Math_Score_by_Grade["11th"]=Math_Score_by_Grade["11th"].map("{:.2f}".format)
Math_Score_by_Grade["12th"]=Math_Score_by_Grade["12th"].map("{:.2f}".format)
Math_Score_by_Grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [21]:
#using condition, getting average value on reading score  
ninth_grade = student[student['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_grade = student[student['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_grade = student[student['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_grade = student[student['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

Reading_Score_by_Grade = pd.DataFrame({"9th": ninth_grade,
                                    "10th": tenth_grade,
                                    "11th": eleventh_grade,
                                    "12th": twelfth_grade,        
                            })
#format
Reading_Score_by_Grade["9th"]=Reading_Score_by_Grade["9th"].map("{:.2f}".format)
Reading_Score_by_Grade["10th"]=Reading_Score_by_Grade["10th"].map("{:.2f}".format)
Reading_Score_by_Grade["11th"]=Reading_Score_by_Grade["11th"].map("{:.2f}".format)
Reading_Score_by_Grade["12th"]=Reading_Score_by_Grade["12th"].map("{:.2f}".format)
Reading_Score_by_Grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [25]:
#create bins 
bins = [0,584,629,644,675]
# default inclusive right value, 
bin_name= ["<$584", "$585-629", "$630-644", "$645-675"]
#adding a column as per bin into the school dataframe 
school["Average Spending" ]= pd.cut(school["Per Student Budget"], bins, labels=bin_name)
#merge updated school dataframe ( with bins column ) and student dataframe
combine= pd.merge(school, student, how="left", on=["school_name", "school_name"])
#create new datafrom after groubby of average spending 
average_math_score=combine.groupby('Average Spending')['math_score'].mean()
average_reading_score=combine.groupby('Average Spending')['reading_score'].mean()

math_pass_rate=combine[combine['math_score'] >= 70].groupby('Average Spending')['Student ID'].count()/combine.groupby('Average Spending')['Student ID'].count() * 100
reading_pass_rate=combine[combine['reading_score'] >= 70].groupby('Average Spending')['Student ID'].count()/combine.groupby('Average Spending')['Student ID'].count() * 100
total_pass_rate=combine[(combine['math_score']>=70) & (combine['reading_score']>=70)].groupby('Average Spending')['Student ID'].count()/combine.groupby('Average Spending')['Student ID'].count() * 100

Score_by_Scool_spending = pd.DataFrame({"Average Math Score": average_math_score,
                        "Average Reading Score": average_reading_score,
                        "Average Reading Score": average_reading_score,
                        "% Passing Math": math_pass_rate,
                        "% Passing Reading": reading_pass_rate,
                        "% Overall Passing": total_pass_rate,        
                            })

Score_by_Scool_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Average Spending,,,,,
<$584,83.363065,83.964039,93.702889,96.686558,90.640704
$585-629,79.982873,82.312643,79.109851,88.513145,70.939239
$630-644,77.821056,81.301007,70.623565,82.600247,58.841194
$645-675,77.049297,81.005604,66.230813,81.109397,53.528791


## Scores by School Size

* Perform the same operations as above, based on school size.

In [28]:
#create bins 
bins = [0, 1000, 2000, 5000]
#name the bins 
bin_name= ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
#adding a column as per bin into the school dataframe 
school["School Size Range" ]= pd.cut(school["size"], bins, labels=bin_name, include_lowest=True)
#merge updated school dataframe ( with bins) and student dataframe
combine= pd.merge(school, student, how="left", on=["school_name", "school_name"])
#create new datafrom after groubby of average spending 
average_math_score=combine.groupby('School Size Range')['math_score'].mean()
average_reading_score=combine.groupby('School Size Range')['reading_score'].mean()

math_pass_rate=combine[combine['math_score'] >= 70].groupby('School Size Range')['Student ID'].count()/combine.groupby('School Size Range')['Student ID'].count() * 100
reading_pass_rate=combine[combine['reading_score'] >= 70].groupby('School Size Range')['Student ID'].count()/combine.groupby('School Size Range')['Student ID'].count() * 100
total_pass_rate=combine[(combine['math_score']>=70) & (combine['reading_score']>=70)].groupby('School Size Range')['Student ID'].count()/combine.groupby('School Size Range')['Student ID'].count() * 100

Score_by_Scool_Size = pd.DataFrame({"Average Math Score": average_math_score,
                        "Average Reading Score": average_reading_score,
                        "Average Reading Score": average_reading_score,
                        "% Passing Math": math_pass_rate,
                        "% Passing Reading": reading_pass_rate,
                        "% Overall Passing": total_pass_rate,        
                            })

Score_by_Scool_Size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size Range,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,56.574046


## Scores by School Type

* Perform the same operations as above, based on school type

In [29]:
#use the intial joined table 

#statistic analysis on student csv
average_math_score=complete.groupby('type')['math_score'].mean()
average_reading_score=complete.groupby('type')['reading_score'].mean()

math_pass_rate=complete[complete['math_score'] >= 70].groupby('type')['Student ID'].count()/complete.groupby('type')['Student ID'].count() * 100
reading_pass_rate=complete[complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/complete.groupby('type')['Student ID'].count() * 100
total_pass_rate=complete[(complete['math_score']>=70) & (complete['reading_score']>=70)].groupby('type')['Student ID'].count()/complete.groupby('type')['Student ID'].count() * 100

#concating all mathmatical results
Score_by_school_type = pd.DataFrame({"Average Math Score": average_math_score,
                        "Average Reading Score": average_reading_score,
                        "% Passing Math": math_pass_rate,
                        "% Passing Reading": reading_pass_rate,
                        "% Overall Passing": total_pass_rate,        
                            })

Score_by_school_type


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878
